## Finding a good neighborhood for your family in the city of Toronto based on your criteria

### Introduction where you discuss the business problem and who would be interested in this project.

People with kids, coming from suburbs, small towns or other countries to Toronto city looking 
for an accomodation have one common problem i.e. finding a good neighborhood for a home. 
They first criteria is having a good school in the neighborhood. And then other amenities
like Restaurants and Parks etc. 

This problem is faced by families all over the world to varying extents. Most impacted are the families with kids who have to be 
more aware of the neighborhood. In some areas, school is the primary driver while in others public transport may be the primary driver.
So this project is intended to guide one to solve this type of problem. Depending on the location and the influencing drivers 
this project can be modified.


### Data where you describe the data that will be used to solve the problem and the source of the data.

The dataset will be dervied from different sources:
--> Location data for toronto will be used from the previous assignments where we got the postal codes, latitudes and longitudes
for Toronto city.

--> Venue data will be extracted using Foursquare API's using the location latitudes and longitudes.

--> School data will be extracted from a website. Initial idea was to scrape the HTML data from this website for all postal codes. 
But this site provides postal code only as an internal filter without any change in the URL. So finally the approach taken was to 
download data for each postal code one at a time and then merge all the postal codes in one file.
http://ontario.compareschoolrankings.org/elementary/SchoolsByRankLocationName.aspx?schooltype=elementary

### Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.

The aim was to find few of the important venues in a neighborhood that can impact a home buyer or renter, who in our case is a family with kids. 
I didn't consider population, size of neighborhood, crime rate etc for this project.

Data analysis was done to understand :
--> what type of venues should be considered. For families with kids, school is a very important criteria. Next i checked for Hospitals and Parks
and found that there are few around most neighborhoods. I narrowed down my search to Grocery Shops and Restaurants.

--> For schools, using foursquare i could not get the full list of elementary schools along with their ratings in Toronto. So i had to use a website to download data for schools by postal code.

Inferential statistical testing:
--> I wanted to include count as well as the mean rating for restaurants and grocery shops. Using Foursquare i checked for the venue ratings,
but found that most of them don't have a rating on Foursquare. So i chose to only include Restaurant and Park counts in the dataset using foursquare.

I used kmeans clustering to group neighborhoods based on these criteria. School Ratings, Park counts, Restaurant counts were passed to the clustering algorithm to give the results in the form of 4 different clusters. This algorithm was best suited for this kind of clustering problem.

### Results section where you discuss the results.

The idea is to group neighborhoods that have similar characterstics i.e. schools followed by grocery shops and restaurants. 
There are 38 neighborhoods in Toronto containing Toronto in the Borough name. On exploring venues using Foursquare, I see that there are
34 neighborhoods with Parks and 30 neighborhoods with Restaurants in a 500 meter radius around the neighborhood location. For Parks and Restaurants, 
i group by neighborhood to get the counts.

Since the first criteria is School, we narrow down our list to the neighborhoods having good elementary schools. 
For this I check elementary school in all 38 neighborhoods using the school data and find 29 elementary schools with rating >= 8. 
Next i group these schools by neighborhood to get the mean rating. Finally i have 16 neighborhoods with good schools to looks at. 

Now i merge these 16 neighborhoods with the Parks and Restaurants dataset to arrive at our final data for clustering. Among these 16 neighborhoods, 
i have few with either no parks or no restaurants.

Finally I run kmeans clustering on this final dataset with a cluseter size of 4. The kmeans clustering algorithm does a good job in clusetering the neighborhoods
based on schools, parks and restaurants.

### Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

The results show that majority of neighborhoods i.e. 8 out of 16 have the same characterstics. They have good schools, few parks and very few restaurants.
The next two big clusters have 3 neighborhoods each. School ratings are almost the same whereas one has fewer parks and restaurants than the other. The smallest 
cluster has only one neighborhood with a very high school rating and a lot of restaurants. Based on the clusters, one can recommend a neighborhood to a family. 

Some observations:

--> Adding more attributes like restaurant rating would have made more useful for the end user to be able to select neighborhoods with good restaurants. 
    Since Foursquare doesn't have good ratings data in this case, would have liked to use Google or other API's instead.

--> Would have like to scrape the school data from the website, but the process is very complicated


### Conclusion section where you conclude the report.

Observing the clusters, one can choose which neighborhoods to look at for finding a home. 
You can use this process and algorithm in other use cases as well. Ex. to find a good neighborhood for a new restaurant or a new grocery shop. 
Or find a neighborhood with low crime rate and population.

The major takeaway from this project is that one can customize this dataset to add more venues as well as more attributes. For ex, we can chose 
to add Grocery shops or Hospitals to the venue list. Similarly one can choose to include the Restaurant rating along with the count for each neighborhood.
So one can have the results based on their set of criteria.